In [1]:
import torch
import torch.nn as nn
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [22]:
#create dataset
df = datasets.load_breast_cancer()

X, y = df.data, df.target

In [23]:
n_samples, n_features = X.shape

In [24]:
print(n_samples)
print(n_features)

569
30


In [25]:
#Train - Test Split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 42)

In [26]:
print(X_train.shape)
print(X_test.shape)

(455, 30)
(114, 30)


In [27]:
print(y_train.shape)
print(y_test.shape)

(455,)
(114,)


In [28]:
#Scaling dataset
scaler = StandardScaler()

In [29]:
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [30]:
X_train = torch.from_numpy(X_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))

In [31]:
y_train = torch.from_numpy(y_train.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))

In [41]:
y_train = y_train.view(y_train.shape[0], 1)
y_test = y_test.view(y_test.shape[0], 1)

#Creat Classification Model
class Model(nn.Module):
    def __init__(self, n_input_features):
        super(Model, self).__init__()
        self.linear = nn.Linear(n_input_features, 1)
        
    def forward(self, x):
        y_pred = torch.sigmoid(self.linear(x))
        return y_pred
    
model = Model(n_features)

##Loss and optimizer
num_epochs = 100
learning_rate = 0.01
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

##Training loop
for epoch in range(num_epochs):
    # Forward pass and loss
    y_pred = model(X_train)
    loss = criterion(y_pred, y_train)

    # Backward pass and update
    loss.backward()
    optimizer.step()

    # zero grad before new step
    optimizer.zero_grad()

    if (epoch+1) % 10 == 0:
        print(f'epoch: {epoch+1}, loss = {loss.item():.4f}')



epoch: 10, loss = 0.6434
epoch: 20, loss = 0.5254
epoch: 30, loss = 0.4522
epoch: 40, loss = 0.4024
epoch: 50, loss = 0.3660
epoch: 60, loss = 0.3382
epoch: 70, loss = 0.3160
epoch: 80, loss = 0.2978
epoch: 90, loss = 0.2826
epoch: 100, loss = 0.2697


In [36]:
#y_train

In [42]:
with torch.no_grad():
    y_predicted = model(X_test)
    y_predicted_cls = y_predicted.round()
    acc = y_predicted_cls.eq(y_test).sum() / float(y_test.shape[0])
    print(f'accuracy: {acc.item():.4f}')

accuracy: 0.9737
